In [1]:
'''
CS585_Lab3
CS585 Image and Video Computing
Lab 3
--------------
This program introduces the following concepts:
	a) Reading a stream of images from a webcamera, and displaying the video
	b) Skin color detection
	c) Background differencing
	d) Visualizing motion history
--------------
'''
import cv2
import sys
import numpy as np

In [2]:
class HandGestureRecognition:
    def __init__(self):
        pass

    def process_frame(self, frame):
        # Convert to HSV color space for skin detection
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower_skin = np.array([0, 48, 80], dtype=np.uint8)
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)
        mask = cv2.inRange(hsv, lower_skin, upper_skin)

        # Improved Morphological Operations
        kernel = np.ones((3, 3), np.uint8)
        mask = cv2.erode(mask, kernel, iterations=2)
        mask = cv2.dilate(mask, kernel, iterations=4)
        mask = cv2.GaussianBlur(mask, (5, 5), 100)

        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 5000]
            if contours:
                max_contour = max(contours, key=cv2.contourArea)
                epsilon = 0.0005 * cv2.arcLength(max_contour, True)
                approx = cv2.approxPolyDP(max_contour, epsilon, True)

                cv2.drawContours(frame, [max_contour], -1, (0, 255, 0), 2)
                cv2.drawContours(frame, [approx], -1, (255, 0, 0), 3)

                hull = cv2.convexHull(max_contour, returnPoints=False)
                if hull is not None and len(hull) > 3:
                    defects = cv2.convexityDefects(max_contour, hull)
                    if defects is not None:
                        self.classify_gesture(defects, frame, max_contour)

        return frame

    def classify_gesture(self, defects, frame, contour):
        count_defects = 0
        for i in range(defects.shape[0]):
            s, e, f, _ = defects[i][0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * (180 / np.pi)

            if angle <= 90:
                count_defects += 1

        if count_defects == 0:
            cv2.putText(frame, "Closed Fist", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        elif count_defects == 1:
            cv2.putText(frame, "Peace sign", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        elif count_defects == 2:
            cv2.putText(frame, "Rock Star sign", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        elif count_defects == 3:
            cv2.putText(frame, "Okay sign", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        elif count_defects >= 4:
            cv2.putText(frame, "Open Hand", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
            

In [3]:
def main():
    cap = cv2.VideoCapture(0)
    hand_gesture_recognition = HandGestureRecognition()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = hand_gesture_recognition.process_frame(frame)
        cv2.imshow('Hand Gesture Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

: 